In [2]:
import tensorflow as tf
import cupy
import numba
from dask.distributed import Client, LocalCluster, fire_and_forget
import asyncio

In [3]:
import numpy as np

def gather_from_detector():
    return np.random.random((1000, 1000))

@numba.jit
def smooth(x):
    out = np.empty_like(x)
    for i in range(1, x.shape[0] - 1):
        for j in range(1, x.shape[1] - 1):
            out[i, j] = (x[i + -1, j + -1] + x[i + -1, j + 0] + x[i + -1, j + 1] +
                         x[i +  0, j + -1] + x[i +  0, j + 0] + x[i +  0, j + 1] +
                         x[i +  1, j + -1] + x[i +  1, j + 0] + x[i +  1, j + 1]) // 9

    return out

def save(x, filename):
    pass

In [5]:
from dask.distributed import Client, LocalCluster, fire_and_forget

cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:33739 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 33.63 GB


In [6]:
%%time

for i in range(1000):
    img = client.submit(gather_from_detector, pure=False)
    img = client.submit(smooth, img)
    img = client.submit(np.fft.fft2, img)
    future = client.submit(save, img, "file-" + str(i) + "-.dat")
    fire_and_forget(future)

CPU times: user 2.17 s, sys: 205 ms, total: 2.37 s
Wall time: 2.44 s
